# 07. RAG統合マルチエージェント（2026年2月アニメ）

3役エージェントにRAGを統合し、根拠付き回答をUIで確認します。


In [ ]:
# 編集禁止セル
import os
import sys
import json

repo_path = '/content/llm_lab'
if os.path.exists(repo_path):
    !rm -rf {repo_path}
!git clone -b ai_agent https://github.com/akio-kobayashi/llm_lab.git {repo_path}
os.chdir(repo_path)

!pip install -q -U sentence-transformers gradio

if 'src' not in sys.path:
    sys.path.append(os.path.abspath('src'))

from src.agent_tools import make_default_registry
from src.agent_core import RulePlannerCoderCriticAgent, RagAwareAgent
from src.rag_anime import AnimeNewsRAG, ensure_anime_news_file
from src.ui_agent import create_multiagent_ui

DATA_PATH = 'data/anime_202602/anime_news_202602.jsonl'
SEED_PATH = 'data/anime_202602/anime_news_202602_seed.jsonl'

if not os.path.exists(DATA_PATH):
    ensure_anime_news_file(DATA_PATH)
    if os.path.exists(SEED_PATH):
        with open(SEED_PATH, 'r', encoding='utf-8') as sf, open(DATA_PATH, 'a', encoding='utf-8') as df:
            for line in sf:
                if line.strip():
                    df.write(line)

print('セットアップ完了。データ:', DATA_PATH)


## 2026年2月データ更新（手動）

以下のようなレコードを `data/anime_202602/anime_news_202602.jsonl` に追記します。

```json
{"title":"記事タイトル","date":"2026-02-16","source":"https://...","summary":"要約"}
```

推奨参照先:
- https://animeanime.jp/article/2026/02/
- https://anime.eiga.com/program/season/2026/


In [ ]:
# 編集禁止セル
rag = AnimeNewsRAG()
rag.load_jsonl(DATA_PATH)
rag.build_index()

registry = make_default_registry()
base_agent = RulePlannerCoderCriticAgent(registry)
agent = RagAwareAgent(base_agent, rag.search)


def run_pipeline(query: str):
    answer, steps = agent.run(query)
    full_log = '

'.join([f"[{s.role}] {s.observation}" for s in steps])
    return answer, full_log, steps


def run_pipeline_rag(query: str):
    answer, steps, docs = agent.run_with_rag(query, top_k=3)
    full_log = '

'.join([f"[{s.role}] {s.observation}" for s in steps])
    return answer, full_log, steps, docs

ui = create_multiagent_ui(run_pipeline=run_pipeline, run_pipeline_rag=run_pipeline_rag)
ui.launch(share=True, debug=True)
